# Given 

- `api_response` (such as JSON from a web-service)
- `dimensions`
- `kpis`

# Expected

A list of flattened JSON with the names of each dimension listed out and the kpis extracted from the inner-most values. 

```python
def flatten_response(api_response, dimensions, kpis):
    pass
```

In [13]:
dimensions = ["network", "campaign", "agenda", "date"]
kpis = ["clicks", "impressions", "results"]

api_response = {
  "network": [
      {
          "name": "Foo",
          "campaign": [
              {
                  "name": "C1",
                  "agenda": [
                      {
                          "name": "A1",
                          "date": [
                              {
                                "date": "2012-01-02",
                                "values": [1, 2, 3]
                              }
                          ]
                      }
                  ]
              }
          ]
      },
      {
        "name": "Bar",
        "campaign": [
            {
              "name": "C2",
              "agenda": [
                  {
                      "name": "A2",
                      "date": [
                          {
                            "date": "2012-01-01",
                            "values": [4, 5, 6]
                          },
                          {
                            "date": "2012-03-01",
                            "values": [4, 5, 6]
                          }
                      ]
                  },
                  {
                      "name": "A2-2",
                      "date": [
                          {
                            "date": "2012-01-01",
                            "values": [4, 5, 6]
                          },
                          {
                            "date": "2012-03-01",
                            "values": [4, 5, 6]
                          }
                      ]
                  },
              ]
            },
            {
              "name": "C2-2",
                  "agenda": [
                      {
                          "name": "A3",
                          "date": [
                              {
                                "date": "2014-01-01",
                                "values": [-2, 5, 60]
                              },
                              {
                                "date": "2015-03-01",
                                "values": [4, 10, 6]
                              }
                          ]
                      }
                  ]
            }
         ]
      }
  ]
}

In [24]:
from typing import Dict, Union, Sequence

JSONValue = Union[str, int]
JSONNode = Dict[str, JSONValue]
JSONArray = Sequence[JSONNode]

def _flatten_last_dimension(response: JSONNode, dimension: str, kpis: Sequence[str]) -> JSONArray:  
    result: JSONArray = []
    for x in response.get(dimension, []):
        # take a copy of the input and pull out the values into the same object
        # x: JSONNode
        # _x = x.copy()
        # for i, k in enumerate(kpis): 
        #     _x[k] = _x['values'][i]
        # del _x['values']
        
        # Other solution: built a new dict
        _x: JSONNode = {'date': x[dimension]}
        for i, k in enumerate(kpis): 
          _x[k] = x['values'][i]

        result.append(_x)

    
    return result
    

def _flatten_response(dimension_response: JSONNode, dimensions: Sequence[str], dim_idx: int, kpis: Sequence[str]) -> JSONArray:
    result: JSONArray = []

    dim_name: str = dimensions[dim_idx]
    for x in dimension_response.get(dim_name, []):
        name: str = x["name"]
        if dim_idx == len(dimensions) - 2:  # next iteration will be the last dimension
            resp = _flatten_last_dimension(x, dimensions[dim_idx + 1], kpis)
        else:
            resp = _flatten_response(x, dimensions, dim_idx + 1, kpis)

        for r in resp:
            r[dim_name] = name
            result.append(r)

    return result

def flatten_response(api_response: JSONNode, dimensions: Sequence[str], kpis: Sequence[str]):
    return _flatten_response(api_response, dimensions, 0, kpis)


In [25]:
for r in flatten_response(api_response, dimensions, kpis):
    print(r)

{'date': '2012-01-02', 'clicks': 1, 'impressions': 2, 'results': 3, 'agenda': 'A1', 'campaign': 'C1', 'network': 'Foo'}
{'date': '2012-01-01', 'clicks': 4, 'impressions': 5, 'results': 6, 'agenda': 'A2', 'campaign': 'C2', 'network': 'Bar'}
{'date': '2012-03-01', 'clicks': 4, 'impressions': 5, 'results': 6, 'agenda': 'A2', 'campaign': 'C2', 'network': 'Bar'}
{'date': '2012-01-01', 'clicks': 4, 'impressions': 5, 'results': 6, 'agenda': 'A2-2', 'campaign': 'C2', 'network': 'Bar'}
{'date': '2012-03-01', 'clicks': 4, 'impressions': 5, 'results': 6, 'agenda': 'A2-2', 'campaign': 'C2', 'network': 'Bar'}
{'date': '2014-01-01', 'clicks': -2, 'impressions': 5, 'results': 60, 'agenda': 'A3', 'campaign': 'C2-2', 'network': 'Bar'}
{'date': '2015-03-01', 'clicks': 4, 'impressions': 10, 'results': 6, 'agenda': 'A3', 'campaign': 'C2-2', 'network': 'Bar'}
